In [1]:
import panel as pn
import os
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import panel as pn
import plotly.express as px
from dotenv import load_dotenv
from panel.interact import interact
import requests
import numpy as np
import alpaca_trade_api as tradeapi
pn.extension('plotly')
%matplotlib inline

ModuleNotFoundError: No module named 'panel'

In [ ]:
cities = ['PHXRNSA','ATXRSA','BOXRSA','NYXRSA','DAXRSA','SEXRNSA','CHXRSA','MIXRNSA','POXRSA','CEXRSA','WDXRSA','DNXRSA', 'CSUSHPISA']
dfs = []
for city in cities:
    city_url = f"https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id={city}&scale=left&cosd=2010-01-01&coed=2020-05-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2020-08-15&revision_date=2020-08-15&nd=1987-01-01"
    df = pd.read_csv(city_url, index_col='DATE')
    dfs.append(df)
df = pd.concat(dfs, axis=1)
df.rename(columns={'PHXRNSA': 'Phoenix', 'ATXRSA': 'Atlanta', 'BOXRSA': 'Boston', 'NYXRSA': 'New York', 'DAXRSA': 'Dallas', 'SEXRNSA': 'Seattle', 'CHXRSA': 'Chicago', 'MIXRNSA': 'Miami', 'POXRSA': 'Portland', 'CEXRSA': 'Cleveland', 'WDXRSA': 'Washington D.C.', 'DNXRSA': 'Denver', 'CSUSHPISA': 'U.S.A'}, inplace=True)
df

In [ ]:
cities = df.columns.tolist()
def myplot(city):
    return df.hvplot(x='DATE', y=city, rot=90, width = (1000), height = (500), title = 'Housing price data by city over the last 10 years')

In [ ]:
pn.interact(myplot, city=cities)

In [ ]:
#In Order to make an Apples to Apples comparison we must convert the data from FRED to a percent change, the dates not alligning correctly will be dealt with below
usa_housing_market = pd.DataFrame(df, columns=['U.S.A'])
usa_house_pct = usa_housing_market.pct_change()
usa_house_pct.reset_index(inplace = True)
usa_house_pct.head()

In [ ]:
from dotenv import load_dotenv
load_dotenv()
#KEYS
alpaca_api_key = ('PKETZDKWTVEJJ92OHW4G')
alpaca_secret_key = ('pFsAVawMY6IEB82q2CwJKhQZi8dDTDywkNjCb5cW')
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

In [ ]:
ticker = ["SPY"]
timeframe = "1D"
start_date = pd.Timestamp('2010-01-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2020-05-01', tz='America/New_York').isoformat()

#calling 2010's top ten companies by size ticker
df_spy = api.get_barset(
    ticker,
    timeframe,
    limit=None,
    start=start_date,
    end=end_date,
    after=None,
    until=None,
).df

# Only comparing closing value
df_spy = df_spy.drop(
    columns=['open', 'high', 'low', 'volume'],
    level=1
)
#Allows the data to reflect the average of a month and post it to a day at the at the end for comparison against housing data.
df_spy.index = df_spy.index.date
df_spy.reset_index(inplace = True)
df_spy.rename(columns={'index': 'Date'}, inplace= True)
dfspy = df_spy.set_index('Date')
dfspy.index = pd.to_datetime(dfspy.index)
monthly_spy = dfspy['SPY'].resample('M').mean()
monthly_spy

In [ ]:
monthly_spy.hvplot(width = (1000), height = (500), title = 'SPY by month for the last decade')

In [ ]:
#applying the pct_change allows for that apples to apples comparison mentioned above.
spy_pct = monthly_spy.pct_change()
spy_pct.head()

In [ ]:
#Joining the two dataframes to compare a high level look at the stock market to a high level look at the housing market, the dates need to match up. We will stick with the housing dates for clairty, stock data being 1 day off will not affect accuracy.
Housing_vs_Stocks = pd.concat([usa_house_pct.reset_index(drop=True),spy_pct.reset_index(drop=True)], axis=1)
Housing_vs_Stocks = Housing_vs_Stocks.set_index('DATE')
Housing_vs_Stocks.rename(columns={'U.S.A.':'US Housing Maket','close': 'Stock Index'}, inplace= True)
Housing_vs_Stocks.head()

In [ ]:
#From this graph we can see the stock market has outpaced the housing market on average, despite the volatility.
cumulative_returns = (1 + Housing_vs_Stocks).cumprod()
cumulative_returns.hvplot(y=['U.S.A', 'Stock Index'],width = (1000), height = (500), xticks = (10),
             value_label='Observed Cumulative Returns')

In [ ]:
#From the above graph vlatility appears to be the name of the game for the stock market. Lets double check. 
monthly_std = Housing_vs_Stocks.std()
monthly_std_visule = (1 + monthly_std).cumprod()
monthly_std_visule.hvplot.scatter(title= 'Volatility comparison')

In [ ]:
def housing_vs_stock():
    """Who performs better"""
    h_v_s = cumulative_returns.hvplot(
        x="DATE",
        y=['U.S.A', 'Stock Index'],
        title = 'Observed Cumulative Returns',
        width = (1000), 
        height = (500),
    ).opts(yformatter="%.0f")
    return h_v_s

Housing_Market = pn.interact(myplot, city=cities)
Monthly_Market_Analysis = pn.Row(housing_vs_stock())

title = pn.pane.Markdown("Housing VS Market")
welcome = pn.pane.Markdown("A simple dashboard dawning complext questions")

Housing_Market = pn.Tabs(
    ('Housing Sales Prive AVG by Month', Housing_Market),
    ('Housing Price Returns VS The Market', Monthly_Market_Analysis),
(
Dashboard=pn.Column(pn.Row(title), Housing_Market, width=900)
Dashboard.servable()